<a href="https://colab.research.google.com/github/KudakwasheMurungweni/FreshProducePricePrediction/blob/main/Tomato_Price_Prediction_using_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
import csv 
drive.mount('/content/drive', force_remount=True)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score

Mounted at /content/drive


Mounting Image directory

In [18]:
#'/content/drive/MyDrive/Datasets/new_dataset.csv')
image_dir = Path('/content/drive/MyDrive/Datasets/Tomato Dataset')



In [21]:
filepaths = pd.Series(list(image_dir.glob(r'**/*.jpg')), name='Filepath').astype(str)
tomatoes = pd.Series(filepaths.apply(lambda x: os.path.split(os.path.split(x)[0])[1]), name='Estimated Average Price').astype(np.float32)

images = pd.concat([filepaths, tomatoes], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [ ]:
images

,Filepath,Estimated Average Price
0,/content/drive/MyDrive/Datasets/Tomato Images/...,40
1,/content/drive/MyDrive/Datasets/Tomato Images/...,60
2,/content/drive/MyDrive/Datasets/Tomato Images/...,100
3,/content/drive/MyDrive/Datasets/Tomato Images/...,40
4,/content/drive/MyDrive/Datasets/Tomato Images/...,60
...,...,...
556,/content/drive/MyDrive/Datasets/Tomato Images/...,60
557,/content/drive/MyDrive/Datasets/Tomato Images/...,60
558,/content/drive/MyDrive/Datasets/Tomato Images/...,40
559,/content/drive/MyDrive/Datasets/Tomato Images/...,100


In [22]:
image_df = images.sample(500, random_state=1).reset_index(drop=True)

train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

In [23]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [24]:
train_df

,Filepath,Estimated Average Price
13,/content/drive/MyDrive/Datasets/Tomato Dataset...,75.0
61,/content/drive/MyDrive/Datasets/Tomato Dataset...,120.0
298,/content/drive/MyDrive/Datasets/Tomato Dataset...,75.0
39,/content/drive/MyDrive/Datasets/Tomato Dataset...,50.0
360,/content/drive/MyDrive/Datasets/Tomato Dataset...,120.0
...,...,...
255,/content/drive/MyDrive/Datasets/Tomato Dataset...,90.0
72,/content/drive/MyDrive/Datasets/Tomato Dataset...,120.0
396,/content/drive/MyDrive/Datasets/Tomato Dataset...,75.0
235,/content/drive/MyDrive/Datasets/Tomato Dataset...,50.0


In [25]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Estimated Average Price',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Estimated Average Price',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Estimated Average Price',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Found 280 validated image filenames.
Found 70 validated image filenames.
Found 150 validated image filenames.


In [26]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
9/9 [==============================] - 109s 12s/step - loss: 8538.5986 - val_loss: 8065.9473
Epoch 2/100
9/9 [==============================] - 4s 428ms/step - loss: 8326.8076 - val_loss: 7676.6187
Epoch 3/100
9/9 [==============================] - 6s 701ms/step - loss: 7630.3984 - val_loss: 6489.3184
Epoch 4/100
9/9 [==============================] - 4s 437ms/step - loss: 5742.4355 - val_loss: 3591.2385
Epoch 5/100
9/9 [==============================] - 6s 666ms/step - loss: 2230.8027 - val_loss: 539.9785
Epoch 6/100
9/9 [==============================] - 4s 431ms/step - loss: 1128.6896 - val_loss: 953.3621
Epoch 7/100
9/9 [==============================] - 6s 625ms/step - loss: 800.6962 - val_loss: 612.0552
Epoch 8/100
9/9 [==============================] - 4s 436ms/step - loss: 850.8395 - val_loss: 563.3900
Epoch 9/100
9/9 [==============================] - 5s 558ms/step - loss: 743.1787 - val_loss: 586.6515
Epoch 10/100
9/9 [==============================] - 4s 441ms/st

In [27]:
predicted_prices = np.squeeze(model.predict(test_images))
true_prices = test_images.labels

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

r2 = r2_score(true_prices, predicted_prices)
print("Test R^2 Score: {:.5f}".format(r2))

5/5 [==============================] - 37s 9s/step
     Test RMSE: 26.54565
Test R^2 Score: -0.18452


In [ ]:
null_rmse = np.sqrt(np.sum((true_prices - np.mean(true_prices))**2) / len(true_prices))
print("Null/Baseline Model Test RMSE: {:.5f}".format(null_rmse))

Null/Baseline Model Test RMSE: 25.56128
